In [ ]:
import pdfplumber
import pandas as pd
from deep_translator import GoogleTranslator


def translator(a):
    for i in newlines:
        to_translate = i
        translated = GoogleTranslator(source='auto', target='en').translate(i)
        print(translated)

    
pdf = pdfplumber.open("france.pdf")
print(len(pdf.pages))

for i in range(1,len(pdf.pages)):
    print('\n',"\033[1m" +'PAGE'+str(i) +"\033[0m",'\n')
    p1 = pdf.pages[i]
    im = p1.to_image()
    text = p1.extract_text()
    lines = text.split("\n")
   
    newlines = []
    for x in lines:
        if x.strip():
            newlines.append(x)
            
    translator(newlines)
    